In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '5k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

Best results yet with: 

```
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000)
```

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=False,
                      list_features_to_calc=[], # 'curvatures', min_curvature' 'max_curvature'
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.01,
                      non_rigid_beta=50,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=600,
                      feature_smoothing_iterations=1,
                      include_points_as_features=False,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([.1,.1]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000,
                      initial_correspondence_type='hungarian',
                      final_correspondence_type='kd')  #'kd' 'hungarian'

Loaded Mesh 1
Beginning Eigen Decomposition
Eigen values are: 
[-3.98708395e-18  8.39246263e-04  1.63007145e-03  2.12549101e-03
  3.13941439e-03  3.77495258e-03  4.01682329e-03]
Computed spectrum 1
Loaded Mesh 2
Beginning Eigen Decomposition
Eigen values are: 
[-9.19459446e-18  8.31236570e-04  1.64152416e-03  2.11362458e-03
  3.09029787e-03  3.88535401e-03  3.92405051e-03]
Computed spectrum 2


### View mesh features

In [4]:
reg.graph_source.eig_vals

array([0.00083124, 0.00164152, 0.00211362, 0.0030903 , 0.00388535,
       0.00392405])

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [5]:
eig_vec = 0

In [6]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [7]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [8]:
reg.align_maps()


Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  0     
     1	|  1     
     2	|  -2    
     3	|  3     
     4	|  -4    
     5	|  5     
*Negative source values means those eigenvectors were flipped*
 
Number of features (including spectral) used for registartion: 3

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML: -8369.811; 	ML change (error):  8369.811; 	Sigma^2:     0.070; 	Sigma^2 change:     0.107
[                                                                        ]
Iteration:2
ML: -9093.622; 	ML change (error):   723.810; 	Sigma^2:     0.061; 	Sigma^2 change:     0.009
[                                                                        ]
Iteration:3
ML: -9146.995; 	ML change (error):    53.373; 	Sigma^2:     0.052; 	Sigma^2 change:     0.009
[                                                                        ]
Iteration:4
ML: -9216.499; 	ML change (error):    69.505; 	Sigma^2:     0.042; 	Sigm

Iteration:45
ML:-11129.607; 	ML change (error):     0.011; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:46
ML:-11129.617; 	ML change (error):     0.010; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:-11129.625; 	ML change (error):     0.008; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:48
ML:-11129.633; 	ML change (error):     0.007; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:49
ML:-11129.639; 	ML change (error):     0.007; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:50
ML:-11129.645; 	ML change (error):     0.006; 	Sigma^2:     0.001; 	Sigma^2 c

Iteration:96
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[=======================                                                 ]
Iteration:97
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[=======================                                                 ]
Iteration:98
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[=======================                                                 ]
Iteration:99
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[=======================                                                 ]
Iteration:100
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[========================                                                ]
Iteration:101
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2

Iteration:147
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===================================                                     ]
Iteration:148
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===================================                                     ]
Iteration:149
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===================================                                     ]
Iteration:150
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[====================================                                    ]
Iteration:151
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[====================================                                    ]
Iteration:152
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:195
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[==============================================                          ]
Iteration:196
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===============================================                         ]
Iteration:197
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===============================================                         ]
Iteration:198
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===============================================                         ]
Iteration:199
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===============================================                         ]
Iteration:200
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:245
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[==========================================================              ]
Iteration:246
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===========================================================             ]
Iteration:247
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[===========================================================             ]
Iteration:248
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========================================================             ]
Iteration:249
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========================================================             ]
Iteration:250
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:293
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================================  ]
Iteration:294
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================================  ]
Iteration:295
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================================  ]
Iteration:296
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================================================================= ]
Iteration:297
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:    -0.000
[======================================================================= ]
Iteration:298
ML:-11129.687; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

array([ 112,    1,    2, ..., 4984, 4953, 4985])

## View the spectral coordinates

In [9]:
print('After registration, the range of coordinate values for:\n Source coordinates:\n'
      '{}\n Target coordinates are: \n{}'.format(np.ptp(reg.source_spectral_coords, axis=0),
                                                         np.ptp(reg.target_spectral_coords, axis=0)))


After registration, the range of coordinate values for:
 Source coordinates:
[1. 1. 1.]
 Target coordinates are: 
[1.00945855 0.97330145 1.01662924]


The range of the source are 1.0 because we scaled them to be this. The target are slightly different because they have moved while registering to source coordinates. 

In [10]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0,
                                 include_target_coordinates=True,
                                 include_non_rigid_aligned=True,
                                 include_rigid_aligned=False,
                                 include_unaligned=False,
                                 point_set_colors=None)

Viewer(point_set_colors=array([[0.12156863, 0.46666667, 0.7058824 ],
       [1.        , 0.49803922, 0.0549019…

Above is a 3D view of the spectral (or other) coordinates used for registration. The starting idx for coords can be changed by changing starting_spectral_coord (depending on number of coordinates/features used to register meshes). 

In [11]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is the source mesh (geometry 0) and the target mesh (geometry 1). Target is coloured sequentially (0-n_points). Source is colored according to the target idx that each source point was registered to. Therefore, colours from the target should be in anatomically the same location as the corresponding colour on the source mesh. <br>

Pink is at the same point in the range of values for both meshes. So, it's alignment between them helps affirm the good registration. 

In [12]:
reg.view_aligned_smoothed_spectral_coords()

Viewer(point_set_colors=array([[0.8392157 , 0.        , 0.        ],
       [0.54901963, 0.23137255, 1.       …

- Build new mesh (using source points, moved to final positions)
- Colour all meshes based on IDX node corresponds to on target mesh. 

In [13]:
reg.get_source_mesh_transformed()
reg.set_all_mesh_scalars_to_corresp_target_idx()

In [14]:
reg.view_meshes(include_source=True,
                include_target=True,
                include_transformed_target=True
                )

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is overlapping visualization of the three meshes: 
- Original Target
- Original Source
- Transformed Source (to target surface). 

Possible to turn each one off/on, turn to wire mesh, change transparency, and change colours. 